### Sistema de visión para la pantalla LCD del medidor ultrasónico domiciliario

In [1]:
import cv2
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt

### Métodos para calibración
* Obtener las coordenadas de la pantalla LCD para corregir rotaciones 

In [ ]:
imgRaw = cv2.imread("img/COM_1.png")

def clickEvent(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(x, ' ', y)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(imgRaw, str(x) + ',' +
                    str(y), (x,y), font,
                    1, (0, 0, 255), 2)
        cv2.imshow('image', imgRaw)
    if event==cv2.EVENT_RBUTTONDOWN:
        print(x, ' ', y)
        font = cv2.FONT_HERSHEY_SIMPLEX
        b = imgRaw[y, x, 0]
        g = imgRaw[y, x, 1]
        r = imgRaw[y, x, 2]
        cv2.putText(imgRaw, str(b) + ',' +
                    str(g) + ',' + str(r),
                    (x,y), font, 1,
                    (255, 255, 0), 2)
        cv2.imshow('image', imgRaw)

cv2.imshow('image', imgRaw)
cv2.setMouseCallback('image', clickEvent)
cv2.waitKey(0)
cv2.destroyAllWindows()

* Obtener las ROI's de los diversos segementos de la pantalla LCD

In [ ]:
coms = ['COM_1', 'COM_2', 'COM_3', 'COM_4', 'COM_5', 'COM_6', 'COM_7', 'COM_8']

for com in coms:
        imgRaw = cv2.imread(f"img/{com}.png")
        ROI = cv2.selectROIs("selecting ROI's", imgRaw, False, False)
        print(f"{com}:\n", ROI)

#### JSON con la información de la calibración de la imagen en el banco de pruebas

In [3]:
jsonJSON = {
  "lcd_coords": [(285, 194), (1656, 231), (1643, 810), (272, 787)],
    # "roi_com_1_len": 29,
    # "roi_com_1": [],
    # "roi_com_2_len": 29,
    # "roi_com_2": [],
    # "roi_com_3_len": 30,
    # "roi_com_3": [],
    # "roi_com_4_len": 30,
    # "roi_com_4": [],
    # "roi_com_5_len": 30,
    # "roi_com_5": [],
    # "roi_com_6_len": 30,
    # "roi_com_6": [],
    # "roi_com_7_len": 32,
    # "roi_com_7": [],
    # "roi_com_8_len": 26,
    # "roi_com_8": [],   
}

In [5]:
class LCD:
  lcdJSON = {
    "lcd_coords": [(285, 194), (1656, 231), (1643, 810), (272, 787)],
  }

  def _orderPoints(self, pts: npt.ArrayLike) -> np.ndarray:
    rect = np.zeros((4, 2), dtype = "float32")
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect


  def _fourPointsTransform(self, image: npt.ArrayLike, pts: npt.ArrayLike) -> np.ndarray:
      rect = self._orderPoints(pts)
      (tl, tr, br, bl) = rect
      widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
      widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
      maxWidth = max(int(widthA), int(widthB))
      heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
      heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
      maxHeight = max(int(heightA), int(heightB))
      dst = np.array([
      [0, 0],
      [maxWidth - 1, 0],
      [maxWidth - 1, maxHeight - 1],
      [0, maxHeight - 1]], dtype = "float32")
      M = cv2.getPerspectiveTransform(rect, dst)
      warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
      return warped


  def getLCD(self, img: npt.ArrayLike) -> np.ndarray:
      points = np.array(self.lcdJSON["lcd_coords"], dtype = "float32")
      return self._fourPointsTransform(img, points)

In [7]:
lcd = LCD()
coms = ['COM_1', 'COM_2', 'COM_3', 'COM_4', 'COM_5', 'COM_6', 'COM_7', 'COM_8']

for com in coms:
    imgRaw = cv2.imread(f"img/{com}.png")
    imgLcd = lcd.getLCD(img=imgRaw)
    cv2.imshow('image', imgLcd)
    plt.show()